In [1]:
# Import
import pandas as pd
import requests
import json
import inspect
from lxml import html
from pathlib import Path
from bs4 import BeautifulSoup as bs
import urllib.request
from splinter import Browser

In [2]:
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
# Variables
file_name = "FFS MLB Sheet.xlsx"
ffs_teams_list = ['ARI','ATL','BAL','BOS','CHC','CWS','CIN','CLE','COL','DET','HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SF','SEA','STL','TB','TEX','TOR','WAS']
ffs_teams_list = ['PHI']
auto_fill_url = "on"

In [4]:
# Import FFS MLB Sheet
# Read Player Names
name_cols = "G"
player_list = []
current_year = "2020"

# Loop through each tab to pull list of player names
for team in ffs_teams_list:
    sheet = pd.read_excel(file_name, sheet_name=team, usecols=name_cols, skiprows=2)
    names = sheet['Player']
    # Remove empty rows
    nan_value = float("NaN")
    names.replace("", nan_value, inplace=True)
    names.dropna(inplace=True)
    
    # Write progress to console
    print(f"Processing data for {team}")
    
    #names = ["Andy Young"] #### FOR TESTING ####
    
    # loop through each name in the sheet
    for name in names:
        
        # Remove FFS lingo from name (formatting)
        name = name.replace('RV', '')
        name = name.replace('R1', '')
        name = name.replace(team, '')
        name = name.strip()
        
        # Player Variables
        final_player_name = name
        final_player_dob = ""
        final_player_position = ""
                
        urlname = name.replace(' ','%20')
        url = f"https://www.baseball-reference.com/search/search.fcgi?search={urlname}"
        browser.visit(url)
        html = browser.html
        soup = bs(html, 'html.parser')
        
        # Find the title of the page to determine if you are getting the Search page or a Player page
        title = soup.findAll('title')[0].text
        
        # Search page
        if name not in title:
            # Variables
            player_names = []
            player_links = []
            for div in soup.findAll('div', {'class': 'search-item'}):
                for a in div.findAll('a'):
                    last_year = ""
                    # Link
                    link = a['href']
                    # Split text on whitespace (10)
                    entry = a.text.split('          ')
                    # Player name
                    player_name = entry[0].strip()
                    # Last year played
                    # If multiple years listed, choose last year
                    if '-' in entry[1]:
                        last_year = (entry[1].strip()).replace('(','').replace(')','').split('-')[1]
                        # If only 1 year listed, choose that year
                    else:
                        last_year = (entry[1].strip()).replace('(','').replace(')','')
                    # If last_year is equal to current_year (if player was active this year)
                    if last_year == current_year or last_year == str(int(current_year) - 1):
                        player_names.append(player_name)
                        player_links.append(link)
            # If multiple current player entries found, manually select the correct entry
            if len(player_links) > 1:
                choice = 1
                # Option - Manual Url matching
                if auto_fill_url == "off":
                    count = 0
                    for p in player_links:
                        count += 1
                        print(f"({count}) https://www.baseball-reference.com{p}")    
                    choice = input(f"Multiple entries for {name} {team}. Select:")
                # Set player url from multiple (default is first entry)    
                url = f"https://www.baseball-reference.com{player_links[int(choice) - 1]}"
                browser.visit(url)
                html = browser.html
                soup = bs(html, 'html.parser')
        
        # All Players
        # DOB
        try:
            final_player_dob = soup.findAll(itemprop="birthDate")[0]['data-birth']
        except:
            pass
        
        # Main Position from Player Page
        try:
            pos = soup.findAll(id="meta")[0].findAll('p')[0]
            # Split text on colon
            pos = pos.text.split(':')
            pos = pos[1].strip()
            final_player_position = pos
        except:
            pass
        # Player Position Eligibility
#         try:
#             for tr in soup.findAll('tr', {'id': '2020:standard_fielding'}):
#                 # Positions
#                 player_pos_list = []
#                 for td in tr.findAll('td', {'data-stat': 'pos'}):
#                     player_pos_list.append(td.text)
                
#                 # Games appeared
#                 player_games_list = []
#                 for td in tr.findAll('td',{'data-stat': 'G'}):
#                     player_games_list.append(td.text)
                
#                 count = 0
#                 for amount in player_games_list:
#                     # ESPN position eligibility 
#                     espn_gp = 10
#                     if int(amount) > espn_gp:
#                         final_player_position.append(player_pos_list[count])
#                         final_games_at_position.append(player_games_list[count])
#                     count += 1
#         except:
#             final_player_position.append()
#             final_games_at_position.append()
        
        # Pitchers
        if 'Pitcher' in final_player_position:
            games = ""
            atbats = ""
            runs = ""
            homeruns = ""
            rbi = ""
            sb = ""
            average = ""
            ops = ""
        # Hitters
        if 'Pitcher' not in final_player_position:
            # Hitter stats
            try:
                stats = soup.findAll('tr', {'id': 'batting_standard.2020'})
                games = stats[0].findAll('td',{'data-stat': 'G'})[0].text
                atbats = stats[0].findAll('td',{'data-stat': 'AB'})[0].text
                runs = stats[0].findAll('td',{'data-stat': 'R'})[0].text
                homeruns = stats[0].findAll('td',{'data-stat': 'HR'})[0].text
                rbi = stats[0].findAll('td',{'data-stat': 'RBI'})[0].text
                sb = stats[0].findAll('td',{'data-stat': 'SB'})[0].text
                average = stats[0].findAll('td',{'data-stat': 'batting_avg'})[0].text
                ops = stats[0].findAll('td',{'data-stat': 'onbase_plus_slugging'})[0].text
            except:
                pass
            
        if final_player_position == "":
            games = ""
            atbats = ""
            runs = ""
            homeruns = ""
            rbi = ""
            sb = ""
            average = ""
            ops = ""
            
        
        # Create player object
        player_dict = {
            "name": final_player_name,
            "dob": final_player_dob,
            "url": url,
            "owner": team,
            "position": final_player_position,
            "games": games,
            "atbats": atbats,
            "runs": runs,
            "homeruns": homeruns,
            "rbi": rbi,
            "sb": sb,
            "average": average,
            "ops": ops,
            "owner": team
            
        }
        
        player_list.append(player_dict)

Processing data for PHI


In [5]:
df = pd.DataFrame(player_list)
df.head(20)


,name,dob,url,owner,position,games,atbats,runs,homeruns,rbi,sb,average,ops
0,Yan Gomes,1987-07-19,https://www.baseball-reference.com/search/sear...,PHI,Catcher,30,109,14,4,13,1,.284,.787
1,Yonathan Daza,1994-02-28,https://www.baseball-reference.com/search/sear...,PHI,Centerfielder and Pinch Hitter,30,109,14,4,13,1,.284,.787
2,Yolmer Sanchez,1992-06-29,https://www.baseball-reference.com/search/sear...,PHI,Second Baseman and Third Baseman,11,16,7,1,1,0,.313,1.164
3,Tyler Duffey,,https://www.baseball-reference.com/search/sear...,PHI,,,,,,,,,
4,Ryan Kriedler,,https://www.baseball-reference.com/search/sear...,PHI,,,,,,,,,
5,Pablo Lopez,1996-03-07,https://www.baseball-reference.com/players/l/l...,PHI,Pitcher,,,,,,,,
6,Nick Ahmed,1990-03-15,https://www.baseball-reference.com/search/sear...,PHI,Shortstop,57,199,29,5,29,4,.266,.729
7,Jeremy Jeffress,1987-09-21,https://www.baseball-reference.com/search/sear...,PHI,Pitcher,,,,,,,,
8,Phil Ervin,1992-07-15,https://www.baseball-reference.com/search/sear...,PHI,Outfielder and Pinch Hitter,37,74,10,0,4,1,.149,.481
9,Jose Pujols,1980-01-16,https://www.baseball-reference.com/players/p/p...,PHI,"First Baseman, Leftfielder and Third Baseman",39,152,15,6,25,0,.224,.665


In [6]:
df.to_csv(r'players/owned.csv')

In [7]:
# Hitter stats
stats = soup.findAll('tr', {'id': 'batting_standard.2020'})
games = stats[0].findAll('td',{'data-stat': 'G'})[0].text
atbats = stats[0].findAll('td',{'data-stat': 'AB'})[0].text
runs = stats[0].findAll('td',{'data-stat': 'R'})[0].text
homeruns = stats[0].findAll('td',{'data-stat': 'HR'})[0].text
rbi = stats[0].findAll('td',{'data-stat': 'RBI'})[0].text
sb = stats[0].findAll('td',{'data-stat': 'SB'})[0].text
average = stats[0].findAll('td',{'data-stat': 'batting_avg'})[0].text
ops = stats[0].findAll('td',{'data-stat': 'onbase_plus_slugging'})[0].text
print(dob, games, atbats, runs, homeruns, rbi, sb, average, ops)

IndexError: list index out of range